### GPT4o classifications of mental health and inpatient/outpatient ###

In [ ]:
import os
import pandas as pd
import numpy as np

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
from llmt.llmtools import Prompt, MentalHealth
from llmt.llmtools import process_prompt
from llmt.openai import OpenAI, create_messages

In [ ]:
# Parameters
model = 'gpt-4o'
temperature = 0

# Directories and files
data_dir = os.path.join(os.environ.get('HOME'), 'home_data', 'hcp')
test_file_name = 'hcp-alldata-250413.parquet'
test_file = os.path.join(data_dir, test_file_name)
df = pd.read_parquet(test_file)
# Filter the labeled data
df = df.loc[df['dset'] == 'train'].\
                reset_index(drop=True).\
                astype({'mental_health': int,
                        'inpatient': int,
                        'outpatient': int})


display(df.head())
print(df.shape)

### Prompt development ###

In [ ]:
# Pick an example of a company that provides mental health services
idx = 10
company_id = df.loc[df['mental_health'] == 1, 'id'].values[10]
ser_id = df.loc[df['id'] == company_id].iloc[0]
display(ser_id)

### Mental health services prompt ###

In [ ]:
mental_health_business_definition = Prompt().load(prompt_name='mental_health_definition')
system_prompt = Prompt().load(prompt_name='mental_health_system_prompt')

system_prompt += f' according to the following definition: {mental_health_business_definition}'

# For the user prompt, we want to include the name of the business and the description
def create_user_prompt(name: str, description: str):
    user_prompt = f"""
        The business or provider {name} is described as: {description} 
        Does the business provide mental or behavioral health care services?
        """
    return process_prompt(user_prompt)

In [ ]:
# Run the model for the selected company
user_prompt = create_user_prompt(name=ser_id['name'], description=ser_id['description'])
messages = create_messages(system_prompt=system_prompt, user_prompt=user_prompt)
output = OpenAI().send_messages(messages=messages,
                                model=model,
                                temperature=temperature,
                                response_format=MentalHealth)
print()
print(output)

### In-Patient mental health services prompt ###